In [86]:
import geopandas as gpd
import pandas as pd
import re
import json

In [2]:
# with open("static/data/us-county-boundaries.geojson") as geojson_data:
#     geojson_json = json.load(geojson_data)

In [25]:
# geojson_json.keys()

dict_keys(['type', 'features'])

In [71]:
local_sections_df = pd.read_excel('../LS with Regions - rev.xlsx')

In [72]:
local_sections_df.head()

,Chapter Code,Chapter Name,County,State,Country,Region
0,L200,Akron,PORTAGE,OH,United States,Central
1,L200,Akron,STARK,OH,United States,Central
2,L200,Akron,SUMMIT,OH,United States,Central
3,L400,Alabama,BIBB,AL,United States,Southeast
4,L400,Alabama,BLOUNT,AL,United States,Southeast


In [73]:
local_sections_gdf = gpd.GeoDataFrame(local_sections_df.copy())
local_sections_gdf['County'] = local_sections_gdf['County'].str.title()
local_sections_gdf['State'] = local_sections_gdf['State'].str.upper()
local_sections_gdf.rename(columns={'County':'name', 'State':'stusab'}, inplace=True)
local_sections_gdf['county_state'] = local_sections_gdf['name'] + ', ' + local_sections_gdf['stusab']
local_sections_gdf.head()

,Chapter Code,Chapter Name,name,stusab,Country,Region,county_state
0,L200,Akron,Portage,OH,United States,Central,"Portage, OH"
1,L200,Akron,Stark,OH,United States,Central,"Stark, OH"
2,L200,Akron,Summit,OH,United States,Central,"Summit, OH"
3,L400,Alabama,Bibb,AL,United States,Southeast,"Bibb, AL"
4,L400,Alabama,Blount,AL,United States,Southeast,"Blount, AL"


In [74]:
county_gdf = gpd.read_file("static/data/us-county-boundaries.geojson")
county_gdf['name'] = county_gdf['name'].str.title()
county_gdf['stusab'] = county_gdf['stusab'].str.upper()
county_gdf['county_state'] = county_gdf['name'] + ', ' + county_gdf['stusab']
county_gdf.head()

,intptlat,countyfp_nozero,name,funcstat,intptlon,lsad,stusab,classfp,awater,countyfp,...,countyns,mtfcc,namelsad,statefp,state_name,cbsafp,csafp,metdivfp,geometry,county_state
0,+35.8570011,137,Stone,A,-092.1404819,06,AR,H1,7842161,137,...,00069902,G4020,Stone County,05,Arkansas,None,None,None,"POLYGON ((-92.41528 35.86091, -92.41513 35.866...","Stone, AR"
1,+39.5377097,1,Churchill,A,-118.2641661,06,NV,H1,190748448,001,...,00858638,G4020,Churchill County,32,Nevada,21980,None,None,"POLYGON ((-119.08928 39.60529, -119.08927 39.6...","Churchill, NV"
2,+41.4330028,7,Middlesex,N,-072.5227802,06,CT,H4,180679140,007,...,00212797,G4020,Middlesex County,09,Connecticut,25540,278,None,"POLYGON ((-72.65367 41.43812, -72.65419 41.438...","Middlesex, CT"
3,+39.7835420,39,Decatur,A,-100.4597084,06,KS,H1,1631670,039,...,00484989,G4020,Decatur County,20,Kansas,None,None,None,"POLYGON ((-100.74044 39.82856, -100.74046 39.8...","Decatur, KS"
4,+39.6249458,115,Morgan,A,-081.8616967,06,OH,H1,13868572,115,...,01074070,G4020,Morgan County,39,Ohio,None,None,None,"POLYGON ((-82.03198 39.63773, -82.03197 39.637...","Morgan, OH"


In [76]:
# consider left join here to catch records that did not match
local_sections_county_gdf = local_sections_gdf.merge(county_gdf, on='county_state')
local_sections_county_gdf.head()

,Chapter Code,Chapter Name,name_x,stusab_x,Country,Region,county_state,intptlat,countyfp_nozero,name_y,...,aland,countyns,mtfcc,namelsad,statefp,state_name,cbsafp,csafp,metdivfp,geometry
0,L200,Akron,Portage,OH,United States,Central,"Portage, OH",+41.1689703,133,Portage,...,1262370046,01074079,G4020,Portage County,39,Ohio,10420,184,None,"POLYGON ((-81.39183 41.27665, -81.39183 41.278..."
1,L200,Akron,Stark,OH,United States,Central,"Stark, OH",+40.8141314,151,Stark,...,1490056765,01065576,G4020,Stark County,39,Ohio,15940,184,None,"POLYGON ((-81.42547 40.90666, -81.42514 40.906..."
2,L200,Akron,Summit,OH,United States,Central,"Summit, OH",+41.1218471,153,Summit,...,1069119180,01074088,G4020,Summit County,39,Ohio,10420,184,None,"POLYGON ((-81.68699 41.13596, -81.68697 41.137..."
3,L400,Alabama,Bibb,AL,United States,Southeast,"Bibb, AL",+33.0158929,7,Bibb,...,1612167476,00161529,G4020,Bibb County,01,Alabama,13820,142,None,"POLYGON ((-87.31226 33.08622, -87.31218 33.087..."
4,L400,Alabama,Blount,AL,United States,Southeast,"Blount, AL",+33.9773575,9,Blount,...,1670103866,00161530,G4020,Blount County,01,Alabama,13820,142,None,"POLYGON ((-86.74919 33.99760, -86.74902 33.997..."


In [78]:
print(f'Before dropping duplicates: {len(local_sections_county_gdf)}')
local_sections_county_gdf.drop_duplicates(inplace=True)
print(f'After dropping duplicates: {len(local_sections_county_gdf)}')

Before dropping duplicates: 2529
After dropping duplicates: 2529


In [84]:
local_sections_county_gdf.drop(columns={'name_y','stusab_y'}, inplace=True)
local_sections_county_gdf.rename(columns={'name_x':'county_name', 'stusab_x':'state_code'}, inplace=True)
local_sections_county_gdf.head()

,Chapter Code,Chapter Name,county_name,state_code,Country,Region,county_state,intptlat,countyfp_nozero,funcstat,...,aland,countyns,mtfcc,namelsad,statefp,state_name,cbsafp,csafp,metdivfp,geometry
0,L200,Akron,Portage,OH,United States,Central,"Portage, OH",+41.1689703,133,A,...,1262370046,01074079,G4020,Portage County,39,Ohio,10420,184,None,"POLYGON ((-81.39183 41.27665, -81.39183 41.278..."
1,L200,Akron,Stark,OH,United States,Central,"Stark, OH",+40.8141314,151,A,...,1490056765,01065576,G4020,Stark County,39,Ohio,15940,184,None,"POLYGON ((-81.42547 40.90666, -81.42514 40.906..."
2,L200,Akron,Summit,OH,United States,Central,"Summit, OH",+41.1218471,153,A,...,1069119180,01074088,G4020,Summit County,39,Ohio,10420,184,None,"POLYGON ((-81.68699 41.13596, -81.68697 41.137..."
3,L400,Alabama,Bibb,AL,United States,Southeast,"Bibb, AL",+33.0158929,7,A,...,1612167476,00161529,G4020,Bibb County,01,Alabama,13820,142,None,"POLYGON ((-87.31226 33.08622, -87.31218 33.087..."
4,L400,Alabama,Blount,AL,United States,Southeast,"Blount, AL",+33.9773575,9,A,...,1670103866,00161530,G4020,Blount County,01,Alabama,13820,142,None,"POLYGON ((-86.74919 33.99760, -86.74902 33.997..."


In [87]:
df_cols = list(local_sections_county_gdf.columns)

for i in range(len(df_cols)):
    df_cols[i] =  re.sub('[^0-9a-zA-Z]+', '_', df_cols[i])
    df_cols[i] = df_cols[i].lower()

local_sections_county_gdf.columns = df_cols
local_sections_county_gdf.head()

,chapter_code,chapter_name,county_name,state_code,country,region,county_state,intptlat,countyfp_nozero,funcstat,...,aland,countyns,mtfcc,namelsad,statefp,state_name,cbsafp,csafp,metdivfp,geometry
0,L200,Akron,Portage,OH,United States,Central,"Portage, OH",+41.1689703,133,A,...,1262370046,01074079,G4020,Portage County,39,Ohio,10420,184,None,"POLYGON ((-81.39183 41.27665, -81.39183 41.278..."
1,L200,Akron,Stark,OH,United States,Central,"Stark, OH",+40.8141314,151,A,...,1490056765,01065576,G4020,Stark County,39,Ohio,15940,184,None,"POLYGON ((-81.42547 40.90666, -81.42514 40.906..."
2,L200,Akron,Summit,OH,United States,Central,"Summit, OH",+41.1218471,153,A,...,1069119180,01074088,G4020,Summit County,39,Ohio,10420,184,None,"POLYGON ((-81.68699 41.13596, -81.68697 41.137..."
3,L400,Alabama,Bibb,AL,United States,Southeast,"Bibb, AL",+33.0158929,7,A,...,1612167476,00161529,G4020,Bibb County,01,Alabama,13820,142,None,"POLYGON ((-87.31226 33.08622, -87.31218 33.087..."
4,L400,Alabama,Blount,AL,United States,Southeast,"Blount, AL",+33.9773575,9,A,...,1670103866,00161530,G4020,Blount County,01,Alabama,13820,142,None,"POLYGON ((-86.74919 33.99760, -86.74902 33.997..."


In [88]:
local_sections_county_gdf.to_file("local_sections_county.geojson", driver='GeoJSON')